In [1]:
from flash import FLASHQuadConfig, FLASHQuadModel
import torch
config = FLASHQuadConfig()
model = FLASHQuadModel(config)
model.eval()
input_ids = torch.randint(0, 12000, (4, 128))
with torch.no_grad():
    outputs = model(
        input_ids=input_ids, output_attentions=True, output_hidden_states=True
    )
    print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[ 0.3753, -0.6875, -1.6076,  ..., -0.3942,  1.1974,  1.1923],
         [-0.1625,  0.8348,  1.8773,  ...,  0.4445,  0.5232,  1.7078],
         [ 2.1113,  1.0867,  0.3037,  ..., -0.1736,  0.7612,  1.7222],
         ...,
         [-0.6852,  0.7218, -0.6865,  ..., -0.5073,  0.8717,  0.7833],
         [ 0.0032, -0.3193,  0.7621,  ..., -0.6178,  1.0829,  1.7103],
         [ 0.8200, -0.8049,  0.7853,  ..., -0.7313,  0.1082,  0.4003]],

        [[ 0.2011, -0.1905,  0.3081,  ..., -0.4844, -0.1493,  0.7795],
         [ 1.5964,  0.8578,  0.7130,  ..., -0.6404,  1.0831,  0.8564],
         [ 0.6232,  1.2811, -0.1796,  ..., -0.3130,  0.6512,  2.0113],
         ...,
         [ 0.8324,  0.0239, -0.3911,  ...,  0.3518,  1.2872,  0.6509],
         [-0.1088, -0.0576, -0.2089,  ..., -0.1579,  0.5308,  1.0245],
         [ 1.3067, -0.5883,  0.9087,  ..., -0.1316,  1.8579,  0.9499]],

        [[-0.2128,  0.3026, -0.6403,  ..., -0.7746,  0.6850,  1.7214],
         [ 

In [2]:
import torch
from flash import FLASHQuadForMaskedLM
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("junnyu/roformer_chinese_char_base")
model = FLASHQuadForMaskedLM.from_pretrained("/home/yj/cluedata/roformer-wwm/wwm_flash_small/checkpoint-70000")
model.eval()
text = "天气预报说今天的天[MASK]很好，那么我[MASK]一起去公园玩吧！"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    pt_outputs = model(**inputs).logits[0]

pt_outputs_sentence = "pytorch: "
for i, id in enumerate(tokenizer.encode(text)):
    if id == tokenizer.mask_token_id:
        val,idx = pt_outputs[i].softmax(-1).topk(k=5)
        tokens = tokenizer.convert_ids_to_tokens(idx)
        new_tokens = []
        for v,t in zip(val.cpu(),tokens):
            new_tokens.append(f"{t}+{round(v.item(),4)}")
        pt_outputs_sentence += "[" + "||".join(new_tokens) + "]"
    else:
        pt_outputs_sentence += "".join(
            tokenizer.convert_ids_to_tokens([id], skip_special_tokens=True))
print(pt_outputs_sentence)


pytorch: 天气预报说今天的天[气+0.9887||都+0.0007||天+0.0007||地+0.0007||空+0.0007]很好，那么我[们+0.4775||就+0.4432||也+0.0544||还+0.0031||想+0.0026]一起去公园玩吧！
